<a href="https://colab.research.google.com/github/jodfedlet/TCC/blob/main/TCCII/Twitter/preprocessing_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importando/instalando bibliotecas

In [55]:
!pip install unidecode
!python3 -m spacy download pt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.2 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [56]:

import pandas as pd
import numpy as np
import nltk
import re
import spacy
import matplotlib as mpl
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from google.colab import drive
drive.mount("/content/drive")
nltk.download('all-corpora')
nltk.download('punkt')
nltk.download('rslp')

from nltk.corpus import stopwords
from unidecode import unidecode
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk import regexp_tokenize


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading collection 'all-corpora'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Download

# Carregamento dos tweets até dezembro de 2021

In [57]:
#tweets_orig = pd.read_csv('/content/drive/My Drive/ColabNotebooks/tweets_until_dec_2021.csv', lineterminator='\n', index_col=0)

tweets_orig = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TCC/csv/all_tweets_with_retweets.csv', lineterminator='\n', index_col=0)

In [58]:
tweets_orig.shape

(185348, 4)

### Retornar o tamanho do df, quantidade de palavras e média

#Estatística dos tweets

In [59]:
def tweets_stats(df):
  df['datetime'] = pd.to_datetime(df.datetime, format='%Y-%m-%d %H:%M:%S')

  def get_df_len__count_words__mean(df):
    count_all_tweets = len(df)
    df['count_word'] = df['text'].str.split().str.len()
    count_all_words = df['count_word'].sum()
    mean = round(count_all_words / count_all_tweets, 2)
    return (df, count_all_tweets, count_all_words, mean)

  #data by year
  def plot_by_year(): 
    by_year = df.groupby([df['datetime'].dt.year.rename('Year')]).agg({'tweet_id':['count'], 'count_word': ['sum']}).reset_index()
    by_year.columns = ['Year', 'Count', 'Sum']
    by_year = by_year.assign(Mean_words = lambda x: round(x['Sum'] / x['Count'], 2))
    print(by_year)

  #by year month
  def plot_by_year_and_month():
    by_year_month = df.groupby([df['datetime'].dt.year.rename('Year'), df['datetime'].dt.month.rename('Month')]).agg({'tweet_id':['count'], 'count_word': ['sum']}).reset_index()
    by_year_month.columns = ['Year', 'Month', 'Count', 'Sum']
    by_year_month = by_year_month.assign(Mean_words = lambda x: round(x['Sum'] / x['Count'], 2))
    print(by_year_month)
   
  df, df_len, count_words, mean_words  = get_df_len__count_words__mean(df)
  print('Total: {0}, Count word: {1}, Mean words: {2}'.format(df_len,count_words, mean_words))
  print()
  print('*****Agrupado por ano****')
  plot_by_year()
  print()
  print('*****Agrupado por mês e ano****')
  plot_by_year_and_month()

###Estatística dos tweets antes do pré-processamento:

In [60]:
tweets_stats(tweets_orig)

Total: 185348, Count word: 4542561, Mean words: 24.51

*****Agrupado por ano****
   Year  Count      Sum  Mean_words
0  2018  23691   531047       22.42
1  2019  29739   742523       24.97
2  2020  56375  1317064       23.36
3  2021  62625  1621237       25.89
4  2022  12918   330690       25.60

*****Agrupado por mês e ano****
    Year  Month  Count     Sum  Mean_words
0   2018      1   2564   56315       21.96
1   2018      2   2525   54548       21.60
2   2018      3   1300   28565       21.97
3   2018      4   1298   29096       22.42
4   2018      5   2613   57054       21.83
5   2018      6   1634   37871       23.18
6   2018      7   2412   48864       20.26
7   2018      8   1978   44443       22.47
8   2018      9   1558   36399       23.36
9   2018     10   1909   43484       22.78
10  2018     11   2170   55271       25.47
11  2018     12   1730   39137       22.62
12  2019      1   2532   65964       26.05
13  2019      2   4208  105975       25.18
14  2019      3   2106   

#Quantidade de tweets/palavras


In [61]:
#count_all_tweets = len(tweets_orig)

In [62]:
#tweets_orig['count_word'] = tweets_orig['text'].str.split().str.len()
#count_all_words = tweets_orig['count_word'].sum()

In [63]:
#count_all_words

In [64]:
#mean = count_all_words / count_all_tweets

In [65]:
#mean

In [66]:
tweets_orig.head()

,datetime,tweet_id,user_id,text,count_word
0,2022-05-30 23:45:05+00:00,1531421471146393601,1217204834211958785,@geglobo Paquetá é haitiano,4
1,2022-05-30 23:00:53+00:00,1531410348141592577,1473798922472767488,@BetoLuiz_RU @jonesmanoel_PCB @deccache O Hait...,34
2,2022-05-30 22:57:20+00:00,1531409451122577410,1521912766151417856,O senhor vai investir na educação da Venezuela...,24
3,2022-05-30 22:55:14+00:00,1531408925945434112,1253882714723409920,"@rsallesmma Que nada, na mão de vocês já tá vi...",18
4,2022-05-30 22:55:14+00:00,1531408923277807617,40320768,@JoppertLeonardo @jonesmanoel_PCB @deccache En...,40


In [67]:
tweets_orig.describe()

,tweet_id,user_id,count_word
count,1.853480e+05,1.853480e+05,185348.000000
mean,1.277737e+18,4.790919e+17,24.508282
std,1.505158e+17,5.497380e+17,14.114966
min,9.476356e+17,2.144000e+03,1.000000
25%,1.181666e+18,1.739771e+08,13.000000
50%,1.268305e+18,2.970576e+09,21.000000
75%,1.413515e+18,1.051261e+18,36.000000
max,1.531421e+18,1.530888e+18,105.000000


In [68]:
tweets_orig.text.head().tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???']

In [69]:
tweets_orig.text.tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???',
 '@alice_mader9 kkkkkk e era 2 haitiano, n entendi nada q falavam',
 '@andre_serret Adoro essa cara de "tô cagado" do General Heleno.\nTenho um primo que foi pro Haiti e sempre mando essas fotos pra ele pr

#Pegar os stop words da língua portuguesa usando NLTK

In [70]:
stop_words = set(stopwords.words('portuguese'))

In [71]:
type(stop_words)


set

In [72]:
stop_words

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houveríamos',
 'houvesse',


#Pré-processamento dos tweets

###Função para limpar os tweets

In [73]:
def pre_processing_tweets(tweet):
  
  #remover pontuações
  def remove_punctuation(tweet):
      return re.sub(r'[^\w\s]','',tweet) 

  #remocao de plavras com mais de 3 ocorrencias seguidas
  def mais_de_3_ocorrencias_seguidas(word):  
    let_ant = word[0]
    count = 0
    mais_de_3 = False;
    for w in word:
      count = count + 1 if w == let_ant else 1
      let_ant = w  
      if count > 3: 
        mais_de_3 = True 
        break  
    return mais_de_3   

  #remover acentuações
  def remove_emphasis(token):
        return unidecode(remove_punctuation(token))

  stop_words_list = [remove_emphasis(st_word) for st_word in stop_words]
 
  #conversão de string para minúsculas, remoção de pontuação, palavras que iniciam com @, #, stopwords ou com tamanho < 2 e se forem diferentes de ht.
  def remove_undesirable_words(tweet):
      new_list_ = []
      tweet = ' '.join(str(word).lower() for word in tweet.split() if not word.startswith('@') and not word.startswith('#'))
     
      for token in wordpunct_tokenize(tweet):
         if token in stop_words_list or (len(token) < 3 and token != 'ht') or mais_de_3_ocorrencias_seguidas(token): continue
         new_list_.append(remove_emphasis(token))
      return ' '.join(new_list_)

  #remover urls
  def remove_url(tweet_without_sw):
      return re.sub(r"http\S+", "", tweet_without_sw)

  #remover palavras que contem ou que são números
  def remove_numeric(tweet):
     return ' '.join([token for token in wordpunct_tokenize(tweet) if not token.isdigit() and not any(map(str.isdigit, token))])    

  tweet = remove_url(tweet)
  tweet = remove_undesirable_words(tweet)
  tweet = remove_numeric(tweet)

  return tweet 



###Exemplo de tweet antes da aplicação da função de pré-procesamento

In [74]:
string_example = "o nao @senhor vai #investir, em 2022, na educação da Venezuela, Argentina, Haiti,cuba.e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso ?????. https://t.co/jmVSYbsG2X"

In [75]:
filtered_tweet = pre_processing_tweets(string_example)

###Exemplo de tweet depois da aplicação da função de pré-procesamento

In [76]:
filtered_tweet

'vai educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez'

###Aplicação da função do pré-processamento nos tweets

In [77]:
tweets_pre_proc = tweets_orig.copy()
tweets_pre_proc['text'] = tweets_orig['text'].map(lambda tweet: pre_processing_tweets(tweet))

In [78]:
tweets_orig.text.head().tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???']

In [79]:
tweets_pre_proc.text.head().tolist()


['paqueta haitiano',
 'haiti desgraca fato erros anulam acertos fala sobre guerra ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto saara haiti',
 'entramos brics invadimos haiti mando bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia china inimigos mortais vai funcionar']

###Remover tweets com menos de 3 palavras

In [80]:
tweets_pre_proc = tweets_pre_proc.loc[tweets_pre_proc['text'].str.split().str.len() > 3]

In [81]:
tweets_pre_proc.shape

(171072, 5)

###Remover os tweets duplicados

In [82]:
tweets_pre_proc.text.head().tolist()

['haiti desgraca fato erros anulam acertos fala sobre guerra ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto saara haiti',
 'entramos brics invadimos haiti mando bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia china inimigos mortais vai funcionar',
 'haitiano entendi nada falavam']

In [83]:
tweets_pre_proc.drop_duplicates(subset = ["text"], keep='first', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


###Quantidade de tweets após a remoção dos tweets duplicados

In [84]:
tweets_pre_proc.shape

(149993, 5)

In [85]:
tweets_orig.text.head().tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???']

In [86]:
tweets_pre_proc.text.head().tolist()

['haiti desgraca fato erros anulam acertos fala sobre guerra ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto saara haiti',
 'entramos brics invadimos haiti mando bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia china inimigos mortais vai funcionar',
 'haitiano entendi nada falavam']

###Estatística dos tweets após o pré-processamento:

In [87]:
tweets_stats(tweets_pre_proc)

Total: 149993, Count word: 2134124, Mean words: 14.23

*****Agrupado por ano****
   Year  Count     Sum  Mean_words
0  2018  18169  246796       13.58
1  2019  24413  364436       14.93
2  2020  47434  649606       13.69
3  2021  48644  702999       14.45
4  2022  11333  170287       15.03

*****Agrupado por mês e ano****
    Year  Month  Count     Sum  Mean_words
0   2018      1   1949   25864       13.27
1   2018      2   1832   24361       13.30
2   2018      3   1025   13464       13.14
3   2018      4    981   13426       13.69
4   2018      5   2091   26936       12.88
5   2018      6   1366   18512       13.55
6   2018      7   1859   23125       12.44
7   2018      8   1517   21022       13.86
8   2018      9   1192   16851       14.14
9   2018     10   1410   19898       14.11
10  2018     11   1711   26171       15.30
11  2018     12   1236   17166       13.89
12  2019      1   2083   32136       15.43
13  2019      2   3546   53123       14.98
14  2019      3   1699   25834 

#Lematização e stemização

In [88]:
pln = spacy.load('pt')

###Lematização

In [89]:
string_lem_example = 'capacetes brancos ajudaram haiti pare besteira ajudou haiti fim exercito capacetes brancos resolvem problema presos campos concentracao argentina criou'

In [90]:
def lemmatization(tweet):
  document = pln(tweet)
  return ' '.join([token.lemma_ for token in document])

In [91]:
tweet_lem_example = lemmatization(string_lem_example)

In [92]:
tweet_lem_example

'capacete branco ajudar haiti parir besteira ajudar haiti fim exercitar capacete branco resolver problema preso campo concentracao argentino criar'

In [93]:
tweets_lemm = tweets_pre_proc.copy()
tweets_lemm['text'] = tweets_pre_proc['text'].map(lambda tweet: lemmatization(tweet))

In [94]:
tweets_lemm.text.head().tolist()

['haiti desgraca fato erro anular acerto falir sobrar guerra ucrania vir posicionar bem criticar posturar zelensky',
 'senhor ir investir educacao venezuela argentino haiti cubar etc paises comunista porque aqui brasil senhor nunca fazer',
 'nado mao voces virar misturar desertar saara haiti',
 'entrar brics invadir haiti mandar bush lula dizer ir fortalecer brics fazer palestrar parlamentar europeu este tratar russia chino inimigo mortal ir funcionar',
 'haitiano entender nado falar']

###Stematização ( não será usado na experimentação)

In [95]:
#string_stem_example = 'capacetes brancos ajudaram haiti pare besteira ajudou haiti fim exercito capacetes brancos resolvem problema presos campos concentracao argentina criou'

In [96]:
# def stemming(tweet):
#   document = pln(tweet)
#   stemmer = nltk.stem.RSLPStemmer()
#   return ' '.join([stemmer.stem(token.text) for token in document])

In [97]:
#string_stem_example = stemming(string_stem_example)

In [98]:
# string_stem_example

In [99]:
# tweets_stem = tweets_pre_proc.copy()
# tweets_stem['text'] = tweets_pre_proc['text'].map(lambda tweet: stemming(tweet))

In [100]:
#tweets_stem.text.head().tolist()

#Comparação de tweets originais, pré-processados, lematizados e stematizados

###Tweets originais

In [101]:
tweets_orig.text.head().tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???']

### Tweets pré-processados

In [102]:
tweets_pre_proc.text.head().tolist()

['haiti desgraca fato erros anulam acertos fala sobre guerra ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto saara haiti',
 'entramos brics invadimos haiti mando bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia china inimigos mortais vai funcionar',
 'haitiano entendi nada falavam']

###Tweets lematizados

In [103]:
tweets_lemm.text.head().tolist()

['haiti desgraca fato erro anular acerto falir sobrar guerra ucrania vir posicionar bem criticar posturar zelensky',
 'senhor ir investir educacao venezuela argentino haiti cubar etc paises comunista porque aqui brasil senhor nunca fazer',
 'nado mao voces virar misturar desertar saara haiti',
 'entrar brics invadir haiti mandar bush lula dizer ir fortalecer brics fazer palestrar parlamentar europeu este tratar russia chino inimigo mortal ir funcionar',
 'haitiano entender nado falar']

###Tweets stematizados

In [104]:
#tweets_stem.text.head().tolist()

##Criando o arquivo .csv dos tweets

In [106]:
#df = pd.DataFrame(tweets_pre_proc)
dfl = pd.DataFrame(tweets_lemm)
#df.to_csv('/content/drive/My Drive/Colab Notebooks/TCC/csv/pre_processed_tweets.csv')
dfl.to_csv('/content/drive/My Drive/Colab Notebooks/TCC/csv/tweets_lematizados.csv')